# Archive statistics

In [1]:
import ipywidgets as widgets
widgets.Widget.widget_types
import requests
import geopy

from IPython.display import display

%matplotlib inline


from ipywidgets import interact
from ipywidgets import Label, Layout

from IPython.display import clear_output

import geopandas
import itertools

import os, json, requests, time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import gzip, zipfile

import datacube
from datacube.utils import geometry
from datacube.model import Range
from datetime import datetime
from datetime import timedelta
import shapely
from datacube.api import GridWorkflow
from datacube.utils.geometry import CRS
from shapely.geometry import mapping

import json

dc = datacube.Datacube()

import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors
import numpy as np

In [2]:
products = dc.list_products()['description'].tolist()
products_name = dc.list_products()['name'].tolist()
center = [-26, 132]
zoom = 3

In [3]:
# Create a dict to allow product picker with description
products_name = dc.list_products()['name'].tolist()
product_dict = {}
for idx, val in enumerate(products):
    product_dict[val]= products_name[idx]

In [4]:
from __future__ import print_function

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from traitlets import link

In [5]:
aoi_picker = widgets.Text(value='Canberra',placeholder='update this field',
    description='placename search',
    disabled=False)
def handle_submit(sender):
    global location
    global center
    global zoom
    location = geolocator.geocode(aoi_picker.value)
    center = [location.latitude, location.longitude]
    zoom = 13
    print(center)
aoi_picker.on_submit(handle_submit)

In [6]:
#TODO Replace with datepicker widget when ipywidgets devs sort this out
from_date_picker = widgets.Text(value='2000-12-31',placeholder='update this field',
    description='start date',
    disabled=False)
def handle_from_date(sender):
    print(from_date_picker.value)
from_date_picker.on_submit(handle_from_date)

In [7]:
#TODO Replace with datepicker widget when ipywidgets devs sort this out
to_date_picker = widgets.Text(value='2001-01-01',placeholder='update this field',
    description='end date',
    disabled=False)
def handle_to_date(sender):
    print(to_date_picker.value)
to_date_picker.on_displayed(handle_to_date)

In [8]:
product_picker = widgets.RadioButtons(
    options=products,
    description='product:',
    disabled=False, layout=Layout(width='100%')
)

In [9]:
gqa_products = []
for i in products:
    if 'Level 1' in i:
        gqa_products.append(i)
gqa_product_picker = widgets.RadioButtons(
    options=gqa_products,
    description='product:',
    disabled=False, layout=Layout(width='100%')
)

In [10]:
zoom_picker =widgets.RadioButtons(
    options=[ 'country','regional','city'],
    description='zoom:',
    disabled=False
)
def handle_zoom_select(sender):
    print(zoom_picker.value)
zoom_picker.on_displayed(handle_to_date)

In [11]:
def geojson_grid_count(product_name, query):

    dc = datacube.Datacube()
    gw = datacube.api.GridWorkflow(dc.index, product=product_name)

    cells = gw.list_cells(group_by='solar_day', product=product_name, **query)

    geojson = cells_list_to_featurecollection(cells)
    return(geojson)

def cells_list_to_featurecollection(cells_dict):
    return {'type': 'FeatureCollection',
            'features': [tile_to_geojsonfeature(tile, cell=cell_to_str(cell))
                         for cell, tile in cells_dict.items()]}

def tile_to_geojsonfeature(tile, **props):
    geometry = tile.geobox.extent.to_crs(CRS('EPSG:4326')).__geo_interface__
    feature = {'type': 'Feature',
               'geometry': geometry,
               'properties': dict(count=str(len(tile.sources)),
                                  **props)}
    return feature

def cell_to_str(cell):
    return ', '.join(str(el) for el in cell)

In [12]:
def n_colors(n, colormap=mpl.cm.hot):
    data = np.linspace(0.0,1.0,n)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

In [13]:
drawcontrol = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})

In [14]:
update_map = widgets.Button(description="Update map on click")
label = widgets.Text(layout=Layout(width='20%'))
display(label)
def hover_handler(event=None, id=None, properties=None):
    label.value = properties['count']
m = None
drawcontrol.last_draw['geometry'] = None
def update_map_clicked(button):
    global m
    global zoom
    global drawcontrol
    global data
    
    # clear the map
    if m:
        m.close()
    # collect and set the zoom level
    print(zoom_picker.value)
    if (zoom_picker.value is 'city'):
        zoom = 13
    if (zoom_picker.value is 'regional'):
        zoom = 7
    if (zoom_picker.value is 'country'):
        zoom = 3
        
    m = Map(center=center, zoom=zoom)

    m.add_control(drawcontrol)

    product = product_dict[product_picker.value]
    
    
    if drawcontrol.last_draw['geometry'] is not None:
        list_lat = []
        list_lon = []
        for i in drawcontrol.last_draw['geometry']['coordinates'][0]:
            list_lon.append(i[0])
            list_lat.append(i[1])

        expression = {'time': (from_date_picker.value, to_date_picker.value),
            'lat': (min(list_lat), max(list_lat)),
            'lon': (min(list_lon), max(list_lon)),
            }
        print('Processing:', product, expression)
        
    else:
        expression = {'time': (from_date_picker.value, to_date_picker.value)}
    
        print('Processing:', product, from_date_picker.value, to_date_picker.value)
    
    # Ensure the data request is valid
    try:
        data = geojson_grid_count(product, expression)
    except:
        print('invalid product and query parameters...modify parameters and try again')
        return()
    
    counts = []
    for property in data['features']:
        counts.append(property['properties']['count'])

    n_features = max(set(map(int, counts))) - min(set(map(int, counts)))
    
    if n_features == 0:
        n_features = 1

    colors = n_colors(n_features)
    
    for feature in data['features']:
        feature['properties']['style'] = {'color':colors[int(feature['properties']['count'])- min(set(map(int, counts)))-1], 'weight': 1, 'fillColor':colors[int(feature['properties']['count'])- min(set(map(int, counts))) -1], 'fillOpacity':0.5}
            
            
    layer = GeoJSON(data=data, hover_style={'fillColor': 'red'})
        
    layer.on_hover(hover_handler)
    m.add_layer(layer)
    m.center = center
    #display(m, label)
    display(m)
    clear_output()
display(update_map)
update_map.on_click(update_map_clicked)

In [15]:
scene_gpdf = geopandas.GeoDataFrame.from_file('/g/data/v10/tmp/aoi_wrs2.geojson')

In [16]:
drawcontrol2 = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})

In [17]:
def get_gqa_data(product):

    
    if drawcontrol2.last_draw['geometry'] is not None:
        list_lat = []
        list_lon = []
        for i in drawcontrol2.last_draw['geometry']['coordinates'][0]:
            list_lon.append(i[0])
            list_lat.append(i[1])

        expression = {'time': (from_date_picker.value, to_date_picker.value),
            'lat': (min(list_lat), max(list_lat)),
            'lon': (min(list_lon), max(list_lon)),
            'sat_path': (87,116),
            'sat_row': (67,91)
            }
    else:
        expression = {'time': (from_date_picker.value, to_date_picker.value),
            'sat_path': (87,116),
            'sat_row': (67,91)
            }  


    data = dc.find_datasets(product=product, **expression)
    return(data)

In [18]:
gqa_data = []
for item in gqa_products:
    product = product_dict[item]
    if gqa_data == []:
        gqa_data = get_gqa_data(product)

In [19]:
gqa_slider_list = gqa_data[0].metadata_doc['gqa']['residual'].keys()
gqa_slider = []    
gqa_dict={}

for i in gqa_slider_list:
   
    gqa_slider.append(widgets.FloatRangeSlider(
        value=[0, 1],
        min=-10,
        max=10.0,
        step=0.1,
        description=i,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='i',
        slider_color='white',
        color='black'
    ))
gqa_sliders = widgets.VBox([widgets.VBox(gqa_slider)])

In [20]:
def get_gqa_dict(data):
    gqadict={}
    index = 0
    for doc in data:   
        gcp_count = (doc.metadata_doc['gqa']['final_gcp_count'])
        residual = (doc.metadata_doc['gqa']['residual'])
        path = (doc.metadata_doc['image']['satellite_ref_point_start']['x'])
        row = (doc.metadata_doc['image']['satellite_ref_point_start']['y'])
        path_row= str(path).zfill(3) + str(row).zfill(3)
    
        gqadict[index] = { 'PR': int(path_row), 'center_time': doc.center_time, 'gcp_count': gcp_count}#,

        #get the statistics for each slider
        slider_residual = {}
        for i in gqa_slider_list:

            if type(residual[i]) is dict:

                gqadict[index].update({i: float(residual[i]['xy'])})
            if type(residual[i]) is float:
                gqadict[index].update({i: float(residual[i])})
            
        index = index+1
    return(gqadict)

In [21]:
# select values between slider values
def get_geojson_gqa(scenegpdf, gqadict):
    queryresultgpdf = geopandas.GeoDataFrame
    index = 0
    gqapandas = queryresultgpdf.from_dict(gqadict).transpose()

    for i in gqa_slider_list:
    
        gqapandas = gqapandas[gqapandas[i].between(gqa_slider[index].value[0], gqa_slider[index].value[1], inclusive=True)]
        index = index + 1
    count_gpdf = (gqapandas.groupby('PR').agg(['count'])).reset_index()[['PR','center_time']]
    gqascene = geopandas.GeoDataFrame.merge(scenegpdf, count_gpdf, on='PR', how='inner')
    gqascene = gqascene.rename(columns = {('center_time', 'count'): 'count'})
    maxcount = gqascene['count'].max()
    mincount = gqascene['count'].min()
    geojsongqa = gqascene.to_json()
    geojsongqa = json.loads(geojsongqa)
    return(geojsongqa, mincount, maxcount)

In [22]:
gqa_dict = get_gqa_dict(gqa_data)

In [23]:
save_csv = widgets.Text(value='file.csv',placeholder='enter your destination file path',
    description='CSV export',
    disabled=False)
def handle_save(sender):
    geopandas = gpd.GeoDataFrame.from_features(data['features'])
    geopandas.to_csv(save_csv.value)
    clear_output()
    print('file saved')

save_csv.on_submit(handle_save)

In [24]:
save_geojson = widgets.Text(value='file.geojson',placeholder='enter your destination file path',
    description='GeoJSON export',
    disabled=False)
    
def handle_save_geojson(sender):
    with open(save_geojson.value, 'w') as outfile:
        json.dump(data, outfile)
    print('file saved')

save_geojson.on_submit(handle_save_geojson)

In [25]:
save_shp = widgets.Text(value='file.shp',placeholder='enter your destination file path',
    description='SHP export',
    disabled=False)
def handle_save(sender):
    geopandas = gpd.GeoDataFrame.from_features(data['features'])
    geopandas.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
    geopandas.to_file(save_shp.value, driver='ESRI Shapefile')
    print('file saved')

save_shp.on_submit(handle_save)

In [26]:
update_gqa = widgets.Button(description="Update GQA on click")
gqa_label = widgets.Text(layout=Layout(width='20%'))
display(gqa_label)

def gqa_hover_handler(event=None, id=None, properties=None):
    gqa_label.value = str(properties['count'])
m2 = None
drawcontrol2.last_draw['geometry'] = None
product = None
def update_gqa_clicked(button):
    global m2
    global zoom
    global drawcontrol2
    global gqa_data
    
    # clear the map
    if m2:
        m2.close()
    # collect and set the zoom level
    print(zoom_picker.value)
    if (zoom_picker.value is 'city'):
        zoom = 13
    if (zoom_picker.value is 'regional'):
        zoom = 7
    if (zoom_picker.value is 'country'):
        zoom = 3
       
    m2 = Map(center=center, zoom=zoom)

    m2.add_control(drawcontrol2)
    product = product_dict[gqa_product_picker.value]  
    # Ensure the data request is valid
    
    try:
        gqa_data = get_gqa_data(product)

        gqa_dict = get_gqa_dict(gqa_data)
        
        geojson_gqa, min_count, max_count = get_geojson_gqa(scene_gpdf, gqa_dict)
        
        
    except:
        print('invalid product and query parameters...modify parameters and try again')
        return()
    
    counts = []

    for feature in geojson_gqa['features']:
        
        counts.append(feature['properties']['count'])
        

    n_features = max(set(map(int, counts))) - min(set(map(int, counts)))
    
    if n_features == 0:
        n_features = 1

    colors = n_colors(n_features) 
    
    for feature in geojson_gqa['features']:
        feature['properties']['style'] = {'color':colors[int(feature['properties']['count'])- min(set(map(int, counts)))-1], 'weight': 1, 'fillColor':colors[int(feature['properties']['count'])- min(set(map(int, counts))) -1], 'fillOpacity':0.5}
                       
    gqa_layer = GeoJSON(data=geojson_gqa, hover_style={'fillColor': 'red'})
        
    gqa_layer.on_hover(gqa_hover_handler)
    m2.add_layer(gqa_layer)
    m2.center = center

    display(m2)
    clear_output()
display(update_gqa)
update_gqa.on_click(update_gqa_clicked)

In [27]:
accordion0 = widgets.Accordion(children=[ product_picker,from_date_picker, to_date_picker, aoi_picker,zoom_picker,save_csv, save_geojson, save_shp], description='Count')#,stats_picker ])
accordion0.set_title(0, 'product')
accordion0.set_title(1, 'from date')
accordion0.set_title(2, 'to date')
accordion0.set_title(3, 'placename')
accordion0.set_title(4, 'zoom level')
accordion0.set_title(5, 'Export CSV')
accordion0.set_title(6, 'Export GeoJSON')
accordion0.set_title(7, 'Export SHP')


In [28]:
accordion1 = widgets.Accordion(children=[gqa_product_picker, gqa_sliders], description='GQA')#,stats_picker ])
accordion1.set_title(0, 'product')
accordion1.set_title(1, 'residual')
tab1 = widgets.VBox([update_gqa, accordion1])

In [29]:
# Ipywidget Tabs
children = [ accordion0, tab1]
tab = widgets.Tab(children=children, _titles={0: 'Statistics', 1: 'GQA'})
display(tab)

2001-01-01
2001-01-01
